In [11]:
import pandas as pd
import polars as pl

# Read Data

In [2]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('NEAR_USDT_10h_avg_volatility_base.csv')

# Display the first few rows of the DataFrame
df

,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote,volatility,next_10h_avg_volatility
0,1602748800000,1.0625,1.1872,1.0625,1.1169,1449407,1.655530e+06,9968,766784,8.779915e+05,0.105037,0.042948
1,1602752400000,1.1169,1.1301,1.0778,1.1179,1616432,1.789316e+06,8743,873713,9.684362e+05,0.046279,0.040067
2,1602756000000,1.1176,1.1650,1.1136,1.1570,1270406,1.441663e+06,8310,721173,8.190750e+05,0.044120,0.037814
3,1602759600000,1.1572,1.1575,1.1210,1.1279,481575,5.503787e+05,4446,227307,2.600841e+05,0.031533,0.036343
4,1602763200000,1.1286,1.1812,1.1025,1.1536,1835160,2.107977e+06,9266,934671,1.074642e+06,0.066627,0.031668
...,...,...,...,...,...,...,...,...,...,...,...,...
36426,1733882400000,6.1670,6.4360,6.1450,6.4070,1842808,1.159954e+07,43108,985495,6.194625e+06,0.045214,0.016956
36427,1733886000000,6.4060,6.5010,6.3920,6.4390,1486724,9.601557e+06,30974,813831,5.257918e+06,0.016767,0.017972
36428,1733889600000,6.4400,6.4710,6.3770,6.4330,750535,4.821506e+06,21899,392378,2.521880e+06,0.014526,0.018845
36429,1733893200000,6.4340,6.4610,6.3740,6.4100,735627,4.713947e+06,18913,336143,2.154630e+06,0.013465,0.019306


# Features Engineering

## non-aggregate 4h

In [3]:
df_m = df.copy()

# Calculate Volatility
df_m.drop(columns=['coin','unused', 'closetime'], inplace=True)

df_m['volatility'] = (df_m['highprice'] - df_m['lowprice']) / df_m['lowprice']

df_m['next_4h_avg_volatility'] = (df_m['volatility'].shift(-1) + df_m['volatility'].shift(-2) + df_m['volatility'].shift(-3) + df_m['volatility'].shift(-4)) / 4

df_m

,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote,volatility,next_4h_avg_volatility
0,1602748800000,1.0625,1.1872,1.0625,1.1169,1449407,1.655530e+06,9968,766784,8.779915e+05,0.117365,0.049656
1,1602752400000,1.1169,1.1301,1.0778,1.1179,1616432,1.789316e+06,8743,873713,9.684362e+05,0.048525,0.052932
2,1602756000000,1.1176,1.1650,1.1136,1.1570,1270406,1.441663e+06,8310,721173,8.190750e+05,0.046157,0.053795
3,1602759600000,1.1572,1.1575,1.1210,1.1279,481575,5.503787e+05,4446,227307,2.600841e+05,0.032560,0.050324
4,1602763200000,1.1286,1.1812,1.1025,1.1536,1835160,2.107977e+06,9266,934671,1.074642e+06,0.071383,0.047671
...,...,...,...,...,...,...,...,...,...,...,...,...
36456,1733990400000,7.0790,7.1800,7.0780,7.1800,1252786,8.950482e+06,20675,658195,4.702462e+06,0.014411,0.014886
36457,1733994000000,7.1790,7.2200,7.1460,7.1490,1535789,1.103534e+07,23363,709728,5.101661e+06,0.010355,NaN
36458,1733997600000,7.1490,7.1560,7.0210,7.0380,2065246,1.461731e+07,35347,875286,6.193253e+06,0.019228,NaN
36459,1734001200000,7.0380,7.0510,6.9450,7.0070,1712135,1.197202e+07,32840,797708,5.579540e+06,0.015263,NaN


In [158]:
df_features = df_m.copy()

# Get Rolling 4h data (helped)
df_features['openprice_4h'] = df_features['openprice'].shift(3)
df_features['highprice_4h'] = df_features['highprice'].rolling(window=4).max()
df_features['lowprice_4h'] = df_features['lowprice'].rolling(window=4).min()
df_features['volume_4h'] = df_features['volume'].rolling(window=4).sum()
df_features['quotevolume_4h'] = df_features['quotevolume'].rolling(window=4).sum()
df_features['trades_4h'] = df_features['trades'].rolling(window=4).sum()
df_features['taker_buy_volume_4h'] = df_features['taker_buy_volume'].rolling(window=4).sum()
df_features['taker_buy_quote_4h'] = df_features['taker_buy_quote'].rolling(window=4).sum()
df_features['4h_volatility'] = (df_features['highprice_4h'] - df_features['lowprice_4h']) / df_features['lowprice_4h']
df_features['current_4h_avg_volatility'] = df_features['volatility'].rolling(window=4).mean()

# (kinda help)
df_features['ma_10_current_4h_avg_volatility'] = df_features['current_4h_avg_volatility'].rolling(window=10).mean()
df_features['ma_20_current_4h_avg_volatility'] = df_features['current_4h_avg_volatility'].rolling(window=20).mean()
df_features['ma_50_current_4h_avg_volatility'] = df_features['current_4h_avg_volatility'].rolling(window=50).mean()

# df_features['ma_10_4h_volatility'] = df_features['4h_volatility'].rolling(window=10).mean()
# df_features['ma_20_4h_volatility'] = df_features['4h_volatility'].rolling(window=20).mean()
# df_features['ma_50_4h_volatility'] = df_features['4h_volatility'].rolling(window=50).mean()

# Volume features
# df_features['volume_change'] = df_features['volume'].diff()
# df_features['quotevolume_change'] = df_features['quotevolume'].diff()
# df_features['relative_volume'] = df_features['volume'] / df_features['quotevolume']
# df_features['taker_buy_volume_ratio'] = df_features['taker_buy_volume'] / df_features['volume']
# df_features['taker_buy_quotevolume_ratio'] = df_features['taker_buy_quote'] / df_features['quotevolume']

# lagged (kinda help)
df_features['std_10_volume'] = df_features['volume'].rolling(window=10).std()
df_features['std_20_volume'] = df_features['volume'].rolling(window=20).std()
df_features['std_50_volume'] = df_features['volume'].rolling(window=50).std()

df_features['sum_10_volume'] = df_features['volume'].rolling(window=10).sum()
df_features['sum_20_volume'] = df_features['volume'].rolling(window=20).sum()
df_features['sum_50_volume'] = df_features['volume'].rolling(window=50).sum()

# lagged (kinda help)
df_features['lag_price_change_1h'] = df_features['closeprice'] - df_features['closeprice'].shift(1)
df_features['lag_price_change_2h'] = df_features['closeprice'] - df_features['closeprice'].shift(2)
df_features['lag_price_change_4h'] = df_features['closeprice'] - df_features['closeprice'].shift(4)
df_features['lag_price_change_8h'] = df_features['closeprice'] - df_features['closeprice'].shift(8)

df_features['lag_volume_change_1h'] = df_features['volume'] - df_features['volume'].shift(1)
df_features['lag_volume_change_2h'] = df_features['volume'] - df_features['volume'].shift(2)
df_features['lag_volume_change_4h'] = df_features['volume'] - df_features['volume'].shift(4)
df_features['lag_volume_change_8h'] = df_features['volume'] - df_features['volume'].shift(8)

# Rolling (help a lot)
df_features['ma_10_volatility'] = df_features['volatility'].rolling(window=10).mean()
df_features['ma_20_volatility'] = df_features['volatility'].rolling(window=20).mean()
df_features['ma_50_volatility'] = df_features['volatility'].rolling(window=50).mean()

# df_features['std_10_volatility'] = df_features['volatility'].rolling(window=10).std()
# df_features['std_20_volatility'] = df_features['volatility'].rolling(window=20).std()
# df_features['std_50_volatility'] = df_features['volatility'].rolling(window=50).std()

# df_features['max_10_volatility'] = df_features['volatility'].rolling(window=10).max()
# df_features['max_20_volatility'] = df_features['volatility'].rolling(window=20).max()
# df_features['max_50_volatility'] = df_features['volatility'].rolling(window=50).max()

# (kinda help)
df_features['lag_volatility_change_1h'] = df_features['volatility'] - df_features['volatility'].shift(1)
df_features['lag_volatility_change_2h'] = df_features['volatility'] - df_features['volatility'].shift(2)
df_features['lag_volatility_change_4h'] = df_features['volatility'] - df_features['volatility'].shift(4)
df_features['lag_volatility_change_8h'] = df_features['volatility'] - df_features['volatility'].shift(8)

# (kinda help)
df_features['min_10_volatility'] = df_features['volatility'].rolling(window=10).min()
df_features['min_20_volatility'] = df_features['volatility'].rolling(window=20).min()
df_features['min_50_volatility'] = df_features['volatility'].rolling(window=50).min()

# (kinda help)
df_features['max_10_open'] = df_features['openprice'].rolling(window=10).max()
df_features['min_10_open'] = df_features['openprice'].rolling(window=10).min()
df_features['max_20_open'] = df_features['openprice'].rolling(window=20).max()
df_features['min_20_open'] = df_features['openprice'].rolling(window=20).min()
df_features['max_50_open'] = df_features['openprice'].rolling(window=50).max()
df_features['min_50_open'] = df_features['openprice'].rolling(window=50).min()

# (kinda help)
df_features['max_10_close'] = df_features['closeprice'].rolling(window=10).max()
df_features['min_10_close'] = df_features['closeprice'].rolling(window=10).min()
df_features['max_20_close'] = df_features['closeprice'].rolling(window=20).max()
df_features['min_20_close'] = df_features['closeprice'].rolling(window=20).min()
df_features['max_50_close'] = df_features['closeprice'].rolling(window=50).max()
df_features['min_50_close'] = df_features['closeprice'].rolling(window=50).min()

# df_features['ma_10_high'] = df_features['highprice'].rolling(window=10).mean()
# df_features['ma_20_high'] = df_features['highprice'].rolling(window=20).mean()
# df_features['ma_50_high'] = df_features['highprice'].rolling(window=50).mean()

# df_features['ma_10_low'] = df_features['lowprice'].rolling(window=10).mean()
# df_features['ma_20_low'] = df_features['lowprice'].rolling(window=20).mean()
# df_features['ma_50_low'] = df_features['lowprice'].rolling(window=50).mean()

# # Seasonality
# df_features['year'] = pd.to_datetime(df_features['opentime'], unit='ms').dt.year
# df_features['month'] = pd.to_datetime(df_features['opentime'], unit='ms').dt.month
df_features['day'] = pd.to_datetime(df_features['opentime'], unit='ms').dt.day
df_features['hour'] = pd.to_datetime(df_features['opentime'], unit='ms').dt.hour

# Shadow Length (kinda help)
df_features['high_open_dif'] = df_features['highprice'] - df_features['openprice']
df_features['high_close_dif'] = df_features['highprice'] - df_features['closeprice']
df_features['low_open_dif'] = df_features['openprice'] - df_features['lowprice']
df_features['low_close_dif'] = df_features['closeprice'] - df_features['lowprice']

# # Price Range Features
# df_features['rolling_range_4h'] = df_features['highprice'].rolling(window=4).max() - df_features['lowprice'].rolling(window=4).min()
# df_features['lagged_range_4h'] = df_features['highprice'].shift(4) - df_features['lowprice'].shift(4)

# Price-to-Volume Ratios (kinda help)
df_features['price_volume_ratio'] = (df_features['closeprice'] - df_features['openprice']) / df_features['volume']
df_features['quoteprice_volume_ratio'] = (df_features['quotevolume']) / (df_features['highprice'] - df_features['lowprice'])

# # Momentum Features
# df_features['momentum_4h'] = df_features['closeprice'] - df_features['closeprice'].shift(4)
# df_features['momentum_8h'] = df_features['closeprice'] - df_features['closeprice'].shift(8)
# df_features['volume_momentum_4h'] = df_features['volume'] - df_features['volume'].shift(4)

# # Relative Strength Index (RSI)
# df_features['price_diff'] = df_features['closeprice'].diff()
# df_features['gain'] = df_features['price_diff'].apply(lambda x: x if x > 0 else 0)
# df_features['loss'] = df_features['price_diff'].apply(lambda x: -x if x < 0 else 0)
# df_features['avg_gain'] = df_features['gain'].rolling(window=14).mean()
# df_features['avg_loss'] = df_features['loss'].rolling(window=14).mean()
# df_features['rs'] = df_features['avg_gain'] / df_features['avg_loss']
# df_features['rsi'] = 100 - (100 / (1 + df_features['rs']))

# # ATR (Average True Range)
# df_features['true_range'] = df_features[['highprice', 'closeprice']].max(axis=1) - df_features[['lowprice', 'closeprice']].min(axis=1)
# df_features['atr_4h'] = df_features['true_range'].rolling(window=4).mean()
# df_features['atr_14h'] = df_features['true_range'].rolling(window=14).mean()

# # Mean Reversion
# df_features['close_ma10_diff'] = df_features['closeprice'] - df_features['closeprice'].rolling(window=10).mean()
# df_features['close_ma50_diff'] = df_features['closeprice'] - df_features['closeprice'].rolling(window=50).mean()

# # High/Low Percentages
# df_features['high_percent'] = (df_features['closeprice'] - df_features['lowprice_4h']) / (df_features['highprice_4h'] - df_features['lowprice_4h'])
# df_features['low_percent'] = (df_features['highprice_4h'] - df_features['closeprice']) / (df_features['highprice_4h'] - df_features['lowprice_4h'])

# Time specific
df_features['hour_of_day'] = pd.to_datetime(df_features['opentime'], unit='ms').dt.hour
df_features['is_peak_hour'] = df_features['hour_of_day'].apply(lambda x: 1 if x in [8, 12, 16] else 0)

# # Lagged volatility
# df_features['volatility_ratio_4h'] = df_features['volatility'] / df_features['volatility'].shift(4)
# df_features['volatility_ratio_8h'] = df_features['volatility'] / df_features['volatility'].shift(8)

# Volume Volatility Interaction
df_features['volatility_volume'] = df_features['volatility'] * df_features['volume']
df_features['volatility_quotevolume_ratio'] = df_features['volatility'] / df_features['quotevolume']

# # MACD (Moving Average Convergence Divergence)
# df_features['ema_12'] = df_features['closeprice'].ewm(span=12, adjust=False).mean()
# df_features['ema_26'] = df_features['closeprice'].ewm(span=26, adjust=False).mean()
# df_features['macd'] = df_features['ema_12'] - df_features['ema_26']
# df_features['macd_signal'] = df_features['macd'].ewm(span=9, adjust=False).mean()

# # Skewness and Kurtosis
# df_features['skew_volatility_4h'] = df_features['volatility'].rolling(window=4).skew()
# df_features['kurtosis_volatility_4h'] = df_features['volatility'].rolling(window=4).kurt()

# Bollinger bands
df_features['bollinger_upper_20'] = df_features['closeprice'].rolling(window=20).mean() + (2 * df_features['closeprice'].rolling(window=20).std())
df_features['bollinger_lower_20'] = df_features['closeprice'].rolling(window=20).mean() - (2 * df_features['closeprice'].rolling(window=20).std())
df_features['bollinger_width_20'] = df_features['bollinger_upper_20'] - df_features['bollinger_lower_20']

# # Range Expansion Index
# df_features['rei'] = (df_features['closeprice'] - df_features['lowprice_4h']) / (df_features['highprice_4h'] - df_features['lowprice_4h'])

# # Weighted Hourly Average Volume
# df_features['weighted_avg_volume'] = (df_features['volume'] * df_features['hour']).rolling(window=4).mean()

# Hourly Seasonality Volatility Ratio
df_features['hourly_volatility_ratio'] = df_features['volatility'] / df_features.groupby('day')['volatility'].transform('mean')

# Session-Based Volatility
df_features['session'] = pd.cut(df_features['hour'], bins=[0, 8, 16, 24], labels=[1, 2, 3])
df_features['session_volatility'] = df_features.groupby('session')['volatility'].transform('mean')

# # Rolling Volume-Volatility Correlation
# df_features['vol_vol_corr'] = (
#     df_features['volume']
#     .rolling(window=20)
#     .corr(df_features['volatility'])
# )

# # Volume-Weighted Volatility
# df_features['volume_weighted_volatility'] = (df_features['volatility'] * df_features['volume']).rolling(window=4).mean()

# # Bollinger Band Width
# df_features['bb_width'] = (
#     (df_features['highprice'].rolling(window=20).mean() + 2 * df_features['highprice'].rolling(window=20).std())
#     - (df_features['lowprice'].rolling(window=20).mean() - 2 * df_features['lowprice'].rolling(window=20).std())
# ) / df_features['closeprice'].rolling(window=20).mean()

# # Percentage B
# df_features['percent_b'] = (
#     (df_features['closeprice'] - df_features['lowprice'].rolling(window=20).mean()) /
#     (df_features['highprice'].rolling(window=20).mean() - df_features['lowprice'].rolling(window=20).mean())
# )

# # Directional Movement Index (DMI)
# df_features['dm_plus'] = df_features['highprice'].diff()
# df_features['dm_minus'] = df_features['lowprice'].diff()
# df_features['dmi'] = df_features['dm_plus'].rolling(window=14).mean() - df_features['dm_minus'].rolling(window=14).mean()

# # True Range and Average True Range (ATR)
# df_features['true_range'] = df_features['highprice'] - df_features['lowprice']
# df_features['atr_14'] = df_features['true_range'].rolling(window=14).mean()

# Momentum Change
df_features['momentum_change_10'] = df_features['closeprice'].diff(1).rolling(window=10).sum()
df_features['momentum_change_20'] = df_features['closeprice'].diff(1).rolling(window=20).sum()
df_features['momentum_change_50'] = df_features['closeprice'].diff(1).rolling(window=50).sum()

df_features.dropna(inplace=True)

df_features

,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote,...,volatility_quotevolume_ratio,bollinger_upper_20,bollinger_lower_20,bollinger_width_20,hourly_volatility_ratio,session,session_volatility,momentum_change_10,momentum_change_20,momentum_change_50
52,1602936000000,0.7994,0.8073,0.7857,0.7935,783374,6.238367e+05,6806,388355,3.097979e+05,...,4.406828e-08,0.895006,0.737644,0.157362,1.359021,2,0.021563,0.0398,-0.0826,-0.3635
53,1602939600000,0.7935,0.8040,0.7744,0.7841,1139354,9.015986e+05,9553,612633,4.856943e+05,...,4.239485e-08,0.883007,0.739133,0.143875,1.889538,2,0.021563,-0.0597,-0.1051,-0.3438
54,1602943200000,0.7840,0.8036,0.7752,0.7978,1905833,1.506094e+06,13371,1010611,7.994994e+05,...,2.432498e-08,0.869304,0.743956,0.125348,1.811064,2,0.021563,-0.0327,-0.0888,-0.3558
55,1602946800000,0.7967,0.8130,0.7887,0.7900,984310,7.858549e+05,5701,529272,4.236381e+05,...,3.920596e-08,0.857510,0.747570,0.109940,1.523083,2,0.021563,-0.0217,-0.0818,-0.4292
56,1602950400000,0.7899,0.7977,0.7560,0.7609,1452344,1.125279e+06,10245,605373,4.696621e+05,...,4.901782e-08,0.856856,0.742274,0.114583,2.726738,2,0.021563,-0.0467,-0.0595,-0.4078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,1733976000000,7.0530,7.1250,7.0400,7.0810,1549828,1.097683e+07,29710,682310,4.834212e+06,...,1.099941e-09,7.108749,6.514951,0.593799,0.523106,1,0.019362,0.2830,0.5360,0.8200
36453,1733979600000,7.0820,7.2150,7.0710,7.1270,2039678,1.460077e+07,34191,1225453,8.771590e+06,...,1.394781e-09,7.143164,6.537836,0.605327,0.882318,1,0.019362,0.2750,0.5730,0.8190
36454,1733983200000,7.1270,7.1680,7.0640,7.1220,1633077,1.162963e+07,26984,677472,4.826473e+06,...,1.265950e-09,7.162178,6.577122,0.585056,0.637861,1,0.019362,0.2590,0.5830,0.7250
36455,1733986800000,7.1210,7.1440,7.0670,7.0800,792438,5.630026e+06,17852,347334,2.468128e+06,...,1.935287e-09,7.166396,6.623604,0.542791,0.472062,1,0.019362,0.2030,0.5070,0.6430


## aggregate 4h

In [102]:
df_m = df.copy()

# Calculate Volatility
df_m.drop(columns=['coin','unused', 'closetime'], inplace=True)

df_m['volatility'] = (df_m['highprice'] - df_m['lowprice']) / df_m['lowprice']

# Example: Assuming your DataFrame is named `df_m`
df_m['opentime'] = pd.to_datetime(df_m['opentime'], unit='ms')  # Convert 'opentime' to datetime

# Step 1: Drop initial rows to align with 4-hour intervals
start_time = df_m['opentime'].iloc[0]
start_offset = (start_time.hour % 4) * 3600000 + start_time.minute * 60000 + start_time.second * 1000 + start_time.microsecond // 1000
if start_offset != 0:
    df_m = df_m.iloc[(4 - start_time.hour % 4):]

# Step 2: Set 'opentime' as index
df_m.set_index('opentime', inplace=True)

# Step 3: Aggregate the data every 4 hours
agg_dict = {
    'openprice': 'first',
    'highprice': 'max',
    'lowprice': 'min',
    'closeprice': 'last',
    'volume': 'sum',
    'quotevolume': 'sum',
    'trades': 'sum',
    'taker_buy_volume': 'sum',
    'taker_buy_quote': 'sum',
    'volatility' : 'mean'
}

df_agg = df_m.resample('4h').agg(agg_dict).dropna().reset_index()

# Target
df_agg['next_volatility'] = df_agg['volatility'].shift(-1)

df_agg.rename(columns={'volatility' : '4h_avg_volatility', 'next_volatility' : 'next_4h_avg_volatility'}, inplace=True)

df_agg.dropna(inplace=True)

df_agg


,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote,4h_avg_volatility,next_4h_avg_volatility
0,2020-10-15 08:00:00,1.0625,1.1872,1.0625,1.1279,4817820,5.436888e+06,31467,2588977,2.925587e+06,0.061152,0.050324
1,2020-10-15 12:00:00,1.1286,1.2231,1.1025,1.1777,7234110,8.514717e+06,44982,3741813,4.409185e+06,0.050324,0.035115
2,2020-10-15 16:00:00,1.1772,1.1800,1.1124,1.1609,3082573,3.527578e+06,15450,1317154,1.509746e+06,0.035115,0.018301
3,2020-10-15 20:00:00,1.1614,1.1676,1.1143,1.1220,1350979,1.536829e+06,7032,614980,7.007536e+05,0.018301,0.028176
4,2020-10-16 00:00:00,1.1210,1.1585,1.1000,1.1082,1678252,1.900205e+06,8564,702786,7.972310e+05,0.028176,0.081511
...,...,...,...,...,...,...,...,...,...,...,...,...
9110,2024-12-11 16:00:00,6.8760,6.9250,6.7410,6.8520,5710584,3.899074e+07,103733,2551115,1.742565e+07,0.013953,0.009367
9111,2024-12-11 20:00:00,6.8510,6.9060,6.8250,6.8450,3149226,2.162340e+07,63129,1365533,9.378979e+06,0.009367,0.021838
9112,2024-12-12 00:00:00,6.8450,7.1500,6.7750,7.0540,7133997,4.963662e+07,137048,3651954,2.542302e+07,0.021838,0.014514
9113,2024-12-12 04:00:00,7.0530,7.2150,7.0400,7.0800,6015021,4.283726e+07,108737,2932569,2.090040e+07,0.014514,0.014814


In [103]:
df_features = df_agg.copy()

# Volume features
df_features['volume_change'] = df_features['volume'].diff()
df_features['quotevolume_change'] = df_features['quotevolume'].diff()
df_features['relative_volume'] = df_features['volume'] / df_features['quotevolume']
df_features['taker_buy_volume_ratio'] = df_features['taker_buy_volume'] / df_features['volume']
df_features['taker_buy_quotevolume_ratio'] = df_features['taker_buy_quote'] / df_features['quotevolume']

# ema
df_features['ema_closeprice'] = df_features['closeprice'].ewm(alpha=0.1, adjust=False).mean()
df_features['ema_volume'] = df_features['volume'].ewm(alpha=0.1, adjust=False).mean() 

# lagged
df_features['lag_price_change_1h'] = df_features['closeprice'] - df_features['closeprice'].shift(1)
df_features['lag_price_change_2h'] = df_features['closeprice'] - df_features['closeprice'].shift(2)
df_features['lag_volume_change_1h'] = df_features['volume'] - df_features['volume'].shift(1)
df_features['lag_volume_change_2h'] = df_features['volume'] - df_features['volume'].shift(2)

df_features.dropna(inplace=True)

df_features

,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote,...,quotevolume_change,relative_volume,taker_buy_volume_ratio,taker_buy_quotevolume_ratio,ema_closeprice,ema_volume,lag_price_change_1h,lag_price_change_2h,lag_volume_change_1h,lag_volume_change_2h
2,2020-10-15 16:00:00,1.1772,1.1800,1.1124,1.1609,3082573,3.527578e+06,15450,1317154,1.509746e+06,...,-4.987139e+06,0.873850,0.427290,0.427984,1.135682,4.861761e+06,-0.0168,0.0330,-4151537.0,-1735247.0
3,2020-10-15 20:00:00,1.1614,1.1676,1.1143,1.1220,1350979,1.536829e+06,7032,614980,7.007536e+05,...,-1.990749e+06,0.879069,0.455211,0.455974,1.134314,4.510683e+06,-0.0389,-0.0557,-1731594.0,-5883131.0
4,2020-10-16 00:00:00,1.1210,1.1585,1.1000,1.1082,1678252,1.900205e+06,8564,702786,7.972310e+05,...,3.633753e+05,0.883195,0.418761,0.419550,1.131702,4.227440e+06,-0.0138,-0.0527,327273.0,-1404321.0
5,2020-10-16 04:00:00,1.1086,1.1087,0.9274,0.9359,6440520,6.439931e+06,31286,2614438,2.599064e+06,...,4.539727e+06,1.000091,0.405936,0.403586,1.112122,4.448748e+06,-0.1723,-0.1861,4762268.0,5089541.0
6,2020-10-16 08:00:00,0.9352,0.9770,0.8600,0.9213,7946880,7.273910e+06,59926,3903522,3.578668e+06,...,8.339790e+05,1.092518,0.491202,0.491987,1.093040,4.798561e+06,-0.0146,-0.1869,1506360.0,6268628.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9110,2024-12-11 16:00:00,6.8760,6.9250,6.7410,6.8520,5710584,3.899074e+07,103733,2551115,1.742565e+07,...,-2.788801e+07,0.146460,0.446735,0.446918,6.816768,1.032557e+07,-0.0240,0.2790,-4193301.0,-275364.0
9111,2024-12-11 20:00:00,6.8510,6.9060,6.8250,6.8450,3149226,2.162340e+07,63129,1365533,9.378979e+06,...,-1.736733e+07,0.145640,0.433609,0.433742,6.819591,9.607936e+06,-0.0070,-0.0310,-2561358.0,-6754659.0
9112,2024-12-12 00:00:00,6.8450,7.1500,6.7750,7.0540,7133997,4.963662e+07,137048,3651954,2.542302e+07,...,2.801321e+07,0.143724,0.511909,0.512183,6.843032,9.360542e+06,0.2090,0.2020,3984771.0,1423413.0
9113,2024-12-12 04:00:00,7.0530,7.2150,7.0400,7.0800,6015021,4.283726e+07,108737,2932569,2.090040e+07,...,-6.799359e+06,0.140416,0.487541,0.487902,6.866729,9.025990e+06,0.0260,0.2350,-1118976.0,2865795.0


## non aggregate 10h

In [61]:
df_features = df.copy()

# Calculate Maker Buy Volume
df_features['maker_buy_volume'] = df_features['volume'] - df_features['taker_buy_volume']
df_features['maker_buy_quote'] = df_features['quotevolume'] - df_features['taker_buy_quote']

# Get Rolling 4h data (helped)
df_features['openprice_10h'] = df_features['openprice'].shift(9)
df_features['highprice_10h'] = df_features['highprice'].rolling(window=10).max()
df_features['lowprice_10h'] = df_features['lowprice'].rolling(window=10).min()
df_features['volume_10h'] = df_features['volume'].rolling(window=10).sum()
df_features['quotevolume_10h'] = df_features['quotevolume'].rolling(window=10).sum()
df_features['trades_10h'] = df_features['trades'].rolling(window=10).sum()
df_features['taker_buy_volume_10h'] = df_features['taker_buy_volume'].rolling(window=10).sum()
df_features['taker_buy_quote_10h'] = df_features['taker_buy_quote'].rolling(window=10).sum()
df_features['maker_buy_volume_10h'] = df_features['maker_buy_volume'].rolling(window=10).sum()
df_features['maker_buy_quote_10h'] = df_features['maker_buy_quote'].rolling(window=10).sum()

df_features['10h_volatility'] = (df_features['highprice_10h'] - df_features['lowprice_10h']) / df_features['lowprice_10h']
df_features['current_10h_avg_volatility'] = df_features['volatility'].rolling(window=10).mean()

# lagged (kinda help)
df_features['std_10_volume'] = df_features['volume'].rolling(window=10).std()
df_features['std_20_volume'] = df_features['volume'].rolling(window=20).std()
df_features['std_50_volume'] = df_features['volume'].rolling(window=50).std()

# lagged (kinda help)
df_features['lag_price_change_1h'] = df_features['closeprice'] - df_features['closeprice'].shift(1)
df_features['lag_price_change_2h'] = df_features['closeprice'] - df_features['closeprice'].shift(2)
df_features['lag_price_change_4h'] = df_features['closeprice'] - df_features['closeprice'].shift(4)
df_features['lag_price_change_8h'] = df_features['closeprice'] - df_features['closeprice'].shift(8)

# Rolling ma std max (help a lot)
df_features['ma_50_volatility'] = df_features['volatility'].rolling(window=50).mean()
df_features['ma_100_volatility'] = df_features['volatility'].rolling(window=100).mean()
df_features['ma_200_volatility'] = df_features['volatility'].rolling(window=200).mean()

df_features['std_50_volatility'] = df_features['volatility'].rolling(window=50).std()
df_features['std_100_volatility'] = df_features['volatility'].rolling(window=100).std()
df_features['std_200_volatility'] = df_features['volatility'].rolling(window=200).std()

df_features['max_50_volatility'] = df_features['volatility'].rolling(window=50).max()
df_features['max_100_volatility'] = df_features['volatility'].rolling(window=100).max()
df_features['max_200_volatility'] = df_features['volatility'].rolling(window=200).max()

# Lag volatility change (kinda help)
df_features['lag_volatility_change_10h'] = df_features['volatility'] - df_features['volatility'].shift(10)
df_features['lag_volatility_change_20h'] = df_features['volatility'] - df_features['volatility'].shift(20)
df_features['lag_volatility_change_40h'] = df_features['volatility'] - df_features['volatility'].shift(40)
df_features['lag_volatility_change_80h'] = df_features['volatility'] - df_features['volatility'].shift(80)

# Max Min Rolling Window (kinda help)
df_features['max_10_open'] = df_features['openprice'].rolling(window=10).max()
df_features['min_10_open'] = df_features['openprice'].rolling(window=10).min()
df_features['max_20_open'] = df_features['openprice'].rolling(window=20).max()
df_features['min_20_open'] = df_features['openprice'].rolling(window=20).min()
df_features['max_50_open'] = df_features['openprice'].rolling(window=50).max()
df_features['min_50_open'] = df_features['openprice'].rolling(window=50).min()

df_features['max_10_close'] = df_features['closeprice'].rolling(window=10).max()
df_features['min_10_close'] = df_features['closeprice'].rolling(window=10).min()
df_features['max_20_close'] = df_features['closeprice'].rolling(window=20).max()
df_features['min_20_close'] = df_features['closeprice'].rolling(window=20).min()
df_features['max_50_close'] = df_features['closeprice'].rolling(window=50).max()
df_features['min_50_close'] = df_features['closeprice'].rolling(window=50).min()

# Trades Features
df_features['ma_10_trades'] = df_features['trades'].rolling(window=10).mean()
df_features['ma_20_trades'] = df_features['trades'].rolling(window=20).mean()
df_features['ma_50_trades'] = df_features['trades'].rolling(window=50).mean()

df_features['sum_10_trades'] = df_features['trades'].rolling(window=10).sum()
df_features['sum_20_trades'] = df_features['trades'].rolling(window=20).sum()
df_features['sum_50_trades'] = df_features['trades'].rolling(window=50).sum()

# Shadow Length (kinda help)
df_features['high_open_dif'] = df_features['highprice'] - df_features['openprice']
df_features['high_close_dif'] = df_features['highprice'] - df_features['closeprice']
df_features['low_open_dif'] = df_features['openprice'] - df_features['lowprice']
df_features['low_close_dif'] = df_features['closeprice'] - df_features['lowprice']

# Price-to-Volume Ratios (kinda help)
df_features['price_volume_ratio'] = (df_features['closeprice'] - df_features['openprice']) / df_features['volume']
df_features['quoteprice_volume_ratio'] = (df_features['quotevolume']) / (df_features['highprice'] - df_features['lowprice'])

# Time specific (kinda help)
df_features['day'] = pd.to_datetime(df_features['opentime'], unit='ms').dt.day
df_features['hour'] = pd.to_datetime(df_features['opentime'], unit='ms').dt.hour
df_features['hour_of_day'] = pd.to_datetime(df_features['opentime'], unit='ms').dt.hour
df_features['is_peak_hour'] = df_features['hour_of_day'].apply(lambda x: 1 if x in [8, 12, 16] else 0)

# Volume Volatility Interaction (kinda help)
df_features['volatility_volume'] = df_features['volatility'] * df_features['volume']
df_features['volatility_quotevolume_ratio'] = df_features['volatility'] / df_features['quotevolume']
df_features['ma_10_volatility_volume'] = df_features['volatility_volume'].rolling(window=10).mean()
df_features['ma_20_volatility_volume'] = df_features['volatility_volume'].rolling(window=20).mean()

# Hourly Seasonality Volatility Ratio (kinda help)
df_features['hourly_volatility_ratio'] = df_features['volatility'] / df_features.groupby('day')['volatility'].transform('mean')

# Maker Taker ratio
df_features['taker_maker_buy_volume_ratio'] = df_features['taker_buy_volume'] / df_features['maker_buy_volume']
df_features['taker_maker_buy_quote_ratio'] = df_features['taker_buy_quote'] / df_features['maker_buy_quote']

df_features['sum_10_taker_maker_buy_volume_ratio'] = df_features['taker_buy_volume'].rolling(window=10).sum() / df_features['maker_buy_volume'].rolling(window=10).sum()
df_features['sum_10_taker_maker_buy_quote_ratio'] = df_features['taker_buy_quote'].rolling(window=10).sum() / df_features['maker_buy_quote'].rolling(window=10).sum()

df_features['ma_10_taker_maker_buy_volume_ratio'] = df_features['taker_buy_volume'].rolling(window=10).mean() / df_features['maker_buy_volume'].rolling(window=10).mean()
df_features['ma_10_taker_maker_buy_quote_ratio'] = df_features['taker_buy_quote'].rolling(window=10).mean() / df_features['maker_buy_quote'].rolling(window=10).mean()

df_features['ma_20_taker_maker_buy_volume_ratio'] = df_features['taker_buy_volume'].rolling(window=20).mean() / df_features['maker_buy_volume'].rolling(window=20).mean()
df_features['ma_20_taker_maker_buy_quote_ratio'] = df_features['taker_buy_quote'].rolling(window=20).mean() / df_features['maker_buy_quote'].rolling(window=20).mean()

# ==============================
df_features.dropna(inplace=True)
df_features

,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote,...,ma_20_volatility_volume,hourly_volatility_ratio,taker_maker_buy_volume_ratio,taker_maker_buy_quote_ratio,sum_10_taker_maker_buy_volume_ratio,sum_10_taker_maker_buy_quote_ratio,ma_10_taker_maker_buy_volume_ratio,ma_10_taker_maker_buy_quote_ratio,ma_20_taker_maker_buy_volume_ratio,ma_20_taker_maker_buy_quote_ratio
199,1603465200000,0.7228,0.7272,0.7107,0.7187,777147,5.584407e+05,7140,409710,2.948779e+05,...,10838.370496,1.084154,1.115048,1.118815,1.013976,1.018148,1.013976,1.018148,0.945311,0.948184
200,1603468800000,0.7185,0.7205,0.6882,0.7004,1128090,7.955156e+05,7397,509055,3.588446e+05,...,13304.425658,2.142050,0.822336,0.821773,1.050453,1.055538,1.050453,1.055538,0.939595,0.942890
201,1603472400000,0.7004,0.7145,0.6952,0.7049,490411,3.463863e+05,3883,288789,2.042947e+05,...,13674.256437,1.290673,1.432329,1.437767,1.063045,1.067658,1.063045,1.067658,0.947394,0.949967
202,1603476000000,0.7043,0.7065,0.7023,0.7036,58612,4.125851e+04,770,21842,1.539406e+04,...,13591.699368,0.284052,0.594017,0.595182,1.083471,1.088635,1.083471,1.088635,0.950674,0.953501
203,1603479600000,0.7036,0.7164,0.7031,0.7149,111609,7.921972e+04,900,90443,6.421380e+04,...,13576.635512,0.887068,4.273032,4.279231,1.002243,1.005082,1.002243,1.005082,0.953609,0.955848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36426,1733882400000,6.1670,6.4360,6.1450,6.4070,1842808,1.159954e+07,43108,985495,6.194625e+06,...,117730.644394,2.117175,1.149516,1.146109,0.917692,0.917387,0.917692,0.917387,0.904357,0.903725
36427,1733886000000,6.4060,6.5010,6.3920,6.4390,1486724,9.601557e+06,30974,813831,5.257918e+06,...,116478.005077,0.785102,1.209451,1.210487,0.979464,0.978028,0.979464,0.978028,0.917044,0.916786
36428,1733889600000,6.4400,6.4710,6.3770,6.4330,750535,4.821506e+06,21899,392378,2.521880e+06,...,113165.059587,0.680199,1.095547,1.096647,0.961162,0.960403,0.961162,0.960403,0.926821,0.926949
36429,1733893200000,6.4340,6.4610,6.3740,6.4100,735627,4.713947e+06,18913,336143,2.154630e+06,...,111659.344662,0.630521,0.841443,0.841877,0.939955,0.939355,0.939955,0.939355,0.929343,0.929678


# Import CSV

In [28]:
df_test = pd.read_csv("/home/ubuntu/Rheza/local-share/02_NEAR_USDT/near_test_1.csv")
df_test

,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote,...,rolling_price_spread,moving_avg_crossover,day_of_week,hour_of_day,return_to_volatility_ratio,historical_volatility,uptrend_volume,downtrend_volume,volume_volatility_interaction,quote_momentum_interaction
0,1602936000000,0.7994,0.8073,0.7857,0.7935,783374,6.238367e+05,6806,388355,3.097979e+05,...,0.2274,-0.146228,6,12,-0.268467,1.695386,0,783374,2.153605e+06,-1.129768e+04
1,1602939600000,0.7935,0.8040,0.7744,0.7841,1139354,9.015986e+05,9553,612633,4.856943e+05,...,0.2253,-0.145322,6,13,-0.309924,0.686394,0,1139354,4.354969e+06,-1.942043e+04
2,1602943200000,0.7840,0.8036,0.7752,0.7978,1905833,1.506094e+06,13371,1010611,7.994994e+05,...,0.1947,-0.141476,6,14,0.480461,1.587069,0,1905833,6.982154e+06,-6.882850e+03
3,1602946800000,0.7967,0.8130,0.7887,0.7900,984310,7.858549e+05,5701,529272,4.236381e+05,...,0.1947,-0.135062,6,15,-0.272952,1.610187,0,984310,3.032678e+06,-8.015720e+03
4,1602950400000,0.7899,0.7977,0.7560,0.7609,1452344,1.125279e+06,10245,605373,4.696621e+05,...,0.1805,-0.131576,6,16,-0.665597,2.716583,0,1452344,8.010945e+06,-3.896842e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36400,1733976000000,7.0530,7.1250,7.0400,7.0810,1549828,1.097683e+07,29710,682310,4.834212e+06,...,0.7860,0.331720,4,4,0.328805,1.528993,1549828,0,1.871241e+06,1.891308e+06
36401,1733979600000,7.0820,7.2150,7.0710,7.1270,2039678,1.460077e+07,34191,1225453,8.771590e+06,...,0.8510,0.342840,4,5,0.312015,0.126791,2039678,0,4.153778e+06,2.785827e+06
36402,1733983200000,7.1270,7.1680,7.0640,7.1220,1633077,1.162963e+07,26984,677472,4.826473e+06,...,0.8510,0.354240,4,6,-0.047652,0.358911,1633077,0,2.404304e+06,1.859578e+06
36403,1733986800000,7.1210,7.1440,7.0670,7.0800,792438,5.630026e+06,17852,347334,2.468128e+06,...,0.8500,0.361680,4,7,-0.528430,0.608333,792438,0,8.634177e+05,5.494905e+05


In [29]:
df_test.columns

Index(['opentime', 'openprice', 'highprice', 'lowprice', 'closeprice',
       'volume', 'quotevolume', 'trades', 'taker_buy_volume',
       'taker_buy_quote', 'volatility', 'avg_volatility_4h',
       'next_avg_volatility_4h', 'next_volatility_4h', 'date', 'month',
       'datetime', 'volume_change', 'quote_volume_change', 'relative_volume',
       'taker_buy_volume_ratio', 'taker_buy_quote_volume_ratio',
       'rolling_avg_volume', 'rolling_std_volume', 'rolling_low_volatility',
       'normalized_price_range', 'rolling_avg_volatility',
       'rolling_std_volatility', 'rolling_avg_volatility_4h',
       'rolling_std_volatility_4h', 'percentage_price_change',
       'intraday_volatility', 'ema_closeprice', 'ema_volume',
       'bollinger_upper', 'bollinger_lower', 'lag_price_change_1h',
       'lag_price_change_2h', 'lag_volume_change_1h', 'lag_volume_change_2h',
       'vwap', 'uptrend_indicator', 'downtrend_indicator',
       'momentum_close_10h', 'momentum_volume_10h', 'relative_h

In [ ]:
X = near_df.select([col for col in near_df.columns if col not in ['opentime', 'datetime', 'next_avg_volatility_4h', 'date', 'next_volatility_4h']])
y = near_df['next_avg_volatility_4h']

# ML Baseline (Linear Regression)

## Evaluation Function

In [4]:
import numpy as np
import polars as pl
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Adjust the evaluate_baseline function to accept only y_pred and y_test
def evaluate_baseline(y_pred, y_test):
    # Convert lists to Numpy arrays
    pred = np.array(y_pred)
    test = np.array(y_test)

    # Evaluation Metrics
    me = np.mean(pred - test)  # Mean Error
    mae = mean_absolute_error(test, pred)  # Mean Absolute Error
    mape = np.mean(np.abs((test - pred) / test)) * 100  # Mean Absolute Percentage Error
    mpe = np.mean((test - pred) / test) * 100  # Mean Percentage Error
    rmse = np.sqrt(mean_squared_error(test, pred))  # Root Mean Squared Error

    r = r2_score(test, pred)  # Coefficient of Determination (R-squared)
    min_max_error = np.abs((np.min(pred) - np.min(test)) + 
                            (np.max(pred) - np.max(test)))  # Min-Max Error

    df = pl.DataFrame({
        f'volatility_prediction': pred,
        f'volatility_actual': test
    })

    df = df.with_columns(
        abs(pl.col(f'volatility_prediction') - pl.col(f'volatility_actual')).alias(f'dif_volatility')
    )

    # overall median abs error
    median_abs_err =  df[f'dif_volatility'].median()

    # overall var abs error
    var_abs_err =  df[f'dif_volatility'].var()

    # Results
    results = {
        "Overall Median Absolute Error" : median_abs_err,
        "Overall Variance Absolute Error" : var_abs_err,
        "Mean Error": me,
        "Mean Absolute Error ": mae,
        "Mean Absolute Percentage Error": mape,
        "Mean Percentage Error": mpe,
        "Root Mean Squared Error": rmse,
        "R-squared": r, 
        "Min-Max Error": min_max_error
    }
    
    return results, df

## Sliding Window

In [230]:
import time
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Separate df into features and target
features_df = df_features.drop(['opentime', 'next_10h_avg_volatility'], axis=1)
target_df = df_features['next_10h_avg_volatility']

# Convert to NumPy arrays
X = features_df.values
y = target_df.values

# Specify parameters for the sliding window approach
num_predictions = 1   # Number of rows to predict
gap = 1               # Gap (number of rows to skip after each window)
max_windows = [100]    # Maximum number of windows to process
set_limit = True      # Set this to False to process all windows

# Define list of window sizes
window_sizes = [1000]

# List to store resultsAggregate
for max_window in max_windows:
# Loop through each window size
    for window_size in window_sizes:Aggregate
        print(f'Processing window size: {window_size}')

        # Calculate the number of windows based on dataset size
        num_windows = len(X) - window_size - num_predictions

        # Adjust the starting point if set_limit is True
        start_index = 0
        if set_limit:
            # Only process the latest max_window
            start_index = max(0, len(X) - window_size - num_predictions - max_window)
            num_windows = min(num_windows, max_window)

        # Initialize lists to store actual and predicted values for this window size
        y_vals = []
        y_preds = []

        # Loop through each sliding window with the gap applied
        for window_number in range(start_index, start_index + num_windows, gap):
            
            start = window_number
            end = start + window_size
            X_train = X[start:end]
            y_train = y[start:end]

            X_train_mean = np.mean(X_train, axis=0)
            X_train_std = np.std(X_train, axis=0)
            X_train_normalized = (X_train - X_train_mean) / X_train_std

            # Get the column index for 'cur_avg_vol_4h' from features_df
            close_index = features_df.columns.get_loc('volatility')

            # Normalize y_train using the mean and std of cur_avg_vol_4h
            close_mean = X_train[:, close_index].mean()
            close_std = X_train[:, close_index].std()
            y_train_normalized = (y_train - close_mean) / close_std

            # Prepare validation data for prediction
            X_val = X[end:end + num_predictions]
            y_val = y[end:end + num_predictions]

            # Normalize validation data using the statistics from the training set
            X_val_normalized = (X_val - X_train_mean) / X_train_std

            # Track the start time of the window processing
            start_time = time.time()

            # Initialize and fit the model
            model = LinearRegression()
            model.fit(X_train_normalized, y_train_normalized)

            # Predict on validation data
            y_pred_val = model.predict(X_val_normalized)
            # Denormalize y_val and y_pred_val using the mean and std of cur_avg_vol_4h
            y_pred_val_denorm = y_pred_val * close_std + close_mean

            # Track the end time of the window processing
            end_time = time.time()

            # Append actual and predicted values to the lists
            y_vals.append(y_val[0])  # Assuming a single prediction per window
            y_preds.append(y_pred_val_denormAggregate[0])  # Assuming a single prediction per window

        # Now that all windows for this window_size are processed, evaluate the baseline
        eval_results, _ = evaluate_baseline(y_preds, y_vals)

        # Append the window size and evaluation metrics to the results list
        all_results.append({
            'eval_size' : max_window,
            'window_size': window_size,
            **eval_results
        })

# Convert the results to a DataFrame for further analysis
results_df = pd.DataFrame(all_results)

# Print the results DataFrame
results_df

Processing window size: 1000


,eval_size,window_size,Overall Median Absolute Error,Overall Variance Absolute Error,Mean Error,Mean Absolute Error,Mean Absolute Percentage Error,Mean Percentage Error,Root Mean Squared Error,R-squared,Min-Max Error
0,100,1000,0.00537,0.000048,-0.001059,0.006831,27.62558,-3.27358,0.009704,0.3576,0.01266


## Direct Random Split

In [231]:
import time
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Separate df into features and target
features_df = df_features.drop(['opentime', 'next_10h_avg_volatility'], axis=1)
target_df = df_features['next_10h_avg_volatility']

# Convert to NumPy arrays
X = features_df.values
y = target_df.values

# Define list of train-test split proportions
train_test_ratios = [0.10]  # Example proportions for training set

# List to store results
all_results = []

# Loop through each train-test ratio
for train_ratio in train_test_ratios:
    print(f'Processing train-test split ratio: {train_ratio}')

    # Perform train-test split
    X_train, X_test, y_train, y_test = train_tAggregateest_split(
        X, y, train_size=train_ratio, random_state=42
    )

    # Normalize training data
    X_train_mean = np.mean(X_train, axis=0)
    X_train_std = np.std(X_train, axis=0)
    X_train_normalized = (X_train - X_train_mean) / X_train_std

    # Get the column index for 'volatility' from features_df
    close_index = features_df.columns.get_loc('volatility')

    # Normalize y_train using the mean and std of the 'volatility' column
    close_mean = X_train[:, close_index].mean()
    close_std = X_train[:, close_index].std()
    y_train_normalized = (y_train - close_mean) / close_std

    # Normalize test data using the training set's statistics
    X_test_normalized = (X_test - X_train_mean) / X_train_std

    # Initialize and fit the model
    start_time = time.time()
    model = LinearRegression()
    model.fit(X_train_normalized, y_train_normalized)

    # Predict on the test set
    y_pred_test = model.predict(X_test_normalized)
    # Denormalize predictions
    y_pred_test_denorm = y_pred_test * close_std + close_mean

    # Track the end time of the process
    end_time = time.time()

    # Evaluate the model
    eval_results, _ = evaluate_baseline(y_pred_test_denorm, y_test)

    # Append results to the list
    all_results.append({
        'train_ratio': train_ratio,
        **eval_results
    })

# Convert results to a DataFrame for further analysis
results_df = pd.DataFrame(all_results)

# Print the results DataFrame
results_df


Processing train-test split ratio: 0.1


,train_ratio,Overall Median Absolute Error,Overall Variance Absolute Error,Mean Error,Mean Absolute Error,Mean Absolute Percentage Error,Mean Percentage Error,Root Mean Squared Error,R-squared,Min-Max Error
0,0.1,0.003263,0.000026,0.000163,0.004532,24.351967,-8.813892,0.006845,0.592524,0.009595


## Chronological Split

### Filtering Data

In [54]:
import datetime

# Define start date
start_date = datetime.datetime(2024, 6, 1)  # Use datetime.datetime for timestamp()

# Convert to Unix timestamp
unix_start_date = int(start_date.timestamp())  # Converts to seconds since epoch

# Filter the DataFrame
df_features = df_features[df_features['opentime'] >= unix_start_date*1000]

df_features

,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote,...,ma_20_volatility_volume,hourly_volatility_ratio,taker_maker_buy_volume_ratio,taker_maker_buy_quote_ratio,sum_10_taker_maker_buy_volume_ratio,sum_10_taker_maker_buy_quote_ratio,ma_10_taker_maker_buy_volume_ratio,ma_10_taker_maker_buy_quote_ratio,ma_20_taker_maker_buy_volume_ratio,ma_20_taker_maker_buy_quote_ratio
31792,1717200000000,7.251,7.279,7.243,7.267,433981,3.151872e+06,12560,224960,1633898.454,...,14246.999783,0.282436,1.076255,1.076369,0.924359,0.924577,0.924359,0.924577,0.931149,0.931720
31793,1717203600000,7.266,7.279,7.213,7.241,421731,3.053457e+06,11589,189155,1369571.081,...,14013.689919,0.517798,0.813304,0.813340,0.950144,0.950093,0.950144,0.950093,0.930784,0.931392
31794,1717207200000,7.242,7.267,7.232,7.247,363907,2.637378e+06,9683,181205,1313323.192,...,13801.702384,0.275044,0.991806,0.991895,0.949660,0.949277,0.949660,0.949277,0.927495,0.928068
31795,1717210800000,7.246,7.259,7.217,7.227,451866,3.269226e+06,9799,225167,1628930.172,...,13763.603991,0.330416,0.993242,0.993071,0.936107,0.935816,0.936107,0.935816,0.924674,0.925217
31796,1717214400000,7.226,7.235,7.207,7.223,333891,2.410899e+06,7895,160899,1162082.987,...,13586.032413,0.221008,0.930095,0.930548,0.934313,0.934067,0.934313,0.934067,0.929724,0.930324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36426,1733882400000,6.167,6.436,6.145,6.407,1842808,1.159954e+07,43108,985495,6194625.123,...,117730.644394,2.117175,1.149516,1.146109,0.917692,0.917387,0.917692,0.917387,0.904357,0.903725
36427,1733886000000,6.406,6.501,6.392,6.439,1486724,9.601557e+06,30974,813831,5257917.985,...,116478.005077,0.785102,1.209451,1.210487,0.979464,0.978028,0.979464,0.978028,0.917044,0.916786
36428,1733889600000,6.440,6.471,6.377,6.433,750535,4.821506e+06,21899,392378,2521879.563,...,113165.059587,0.680199,1.095547,1.096647,0.961162,0.960403,0.961162,0.960403,0.926821,0.926949
36429,1733893200000,6.434,6.461,6.374,6.410,735627,4.713947e+06,18913,336143,2154629.652,...,111659.344662,0.630521,0.841443,0.841877,0.939955,0.939355,0.939955,0.939355,0.929343,0.929678


### Lin Reg Model

In [62]:
import time
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Separate df into features and target
features_df = df_features.drop(['opentime', 'next_10h_avg_volatility'], axis=1)
target_df = df_features['next_10h_avg_volatility']

# Convert to NumPy arrays
X = features_df.values
y = target_df.values

# Define list of train-test split proportions
train_test_ratios = [0.93]  # Example proportions for training set

# List to store results
all_results = []

# Loop through each train-test ratio
for train_ratio in train_test_ratios:
    print(f'Processing train-test split ratio: {train_ratio}')

    # Split data chronologically
    split_index = int(len(X) * train_ratio)
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # Normalize training data
    X_train_mean = np.mean(X_train, axis=0)
    X_train_std = np.std(X_train, axis=0)
    X_train_normalized = (X_train - X_train_mean) / X_train_std

    # Get the column index for 'volatility' from features_df
    close_index = features_df.columns.get_loc('volatility')

    # Normalize y_train using the mean and std of the 'volatility' column
    close_mean = X_train[:, close_index].mean()
    close_std = X_train[:, close_index].std()
    y_train_normalized = (y_train - close_mean) / close_std

    # Normalize test data using the training set's statistics
    X_test_normalized = (X_test - X_train_mean) / X_train_std

    # Initialize and fit the model
    start_time = time.time()
    model = LinearRegression()
    model.fit(X_train_normalized, y_train_normalized)

    # Predict on the test set
    y_pred_test = model.predict(X_test_normalized)
    # Denormalize predictions
    y_pred_test_denorm = y_pred_test * close_std + close_mean

    # Track the end time of the process
    end_time = time.time()

    # Evaluate the model
    eval_results, _ = evaluate_baseline(y_pred_test_denorm, y_test)


    # Append results to the list
    all_results.append({
        'train_ratio': train_ratio,
        **eval_results,

    })

# Convert results to a DataFrame for further analysis
results_df = pd.DataFrame(all_results)

# Print the results DataFrame
results_df

Processing train-test split ratio: 0.93


,train_ratio,Overall Median Absolute Error,Overall Variance Absolute Error,Mean Error,Mean Absolute Error,Mean Absolute Percentage Error,Mean Percentage Error,Root Mean Squared Error,R-squared,Min-Max Error
0,0.93,0.002805,0.000011,0.000259,0.003613,22.123225,-8.614784,0.004914,0.523072,0.015151


In [63]:
# Evaluate the model
eval_results, _ = evaluate_baseline(y_pred_test_denorm[-100:], y_test[-100:])

# Append results to the list
all_results.append({
    'train_ratio': train_ratio,
    **eval_results,

})

# Convert results to a DataFrame for further analysis
results_df = pd.DataFrame(all_results)

results_df.T

,0,1
train_ratio,0.930000,0.930000
Overall Median Absolute Error,0.002805,0.004309
Overall Variance Absolute Error,0.000011,0.000058
Mean Error,0.000259,-0.000595
Mean Absolute Error,0.003613,0.006562
Mean Absolute Percentage Error,22.123225,27.786509
Mean Percentage Error,-8.614784,-10.900869
Root Mean Squared Error,0.004914,0.010024
R-squared,0.523072,0.291687
Min-Max Error,0.015151,0.014014


In [64]:
# Define the condition
condition = y_test[-100:] >= 0.02

# Calculate the percentage of values meeting the condition
percentage = (condition.sum() / len(y_test[-100:])) * 100

# Print the result
print(f"Percentage of values >= 0.02: {percentage:.2f}%")

Percentage of values >= 0.02: 50.00%


### Turn value to class

In [65]:
import time
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Separate df into features and target
features_df = df_features.drop(['opentime', 'next_10h_avg_volatility'], axis=1)
target_df = df_features['next_10h_avg_volatility']

# Convert to NumPy arrays
X = features_df.values
y = target_df.values

# Define list of train-test split proportions
train_test_ratios = [0.93]  # Example proportions for training set

# List to store results
all_results = []

# Loop through each train-test ratio
for train_ratio in train_test_ratios:
    print(f'Processing train-test split ratio: {train_ratio}')

    # Split data chronologically
    split_index = int(len(X) * train_ratio)
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # Normalize training data
    X_train_mean = np.mean(X_train, axis=0)
    X_train_std = np.std(X_train, axis=0)
    X_train_normalized = (X_train - X_train_mean) / X_train_std

    # Get the column index for 'volatility' from features_df
    close_index = features_df.columns.get_loc('volatility')

    # Normalize y_train using the mean and std of the 'volatility' column
    close_mean = X_train[:, close_index].mean()
    close_std = X_train[:, close_index].std()
    y_train_normalized = (y_train - close_mean) / close_std

    # Normalize test data using the training set's statistics
    X_test_normalized = (X_test - X_train_mean) / X_train_std

    # Initialize and fit the model
    start_time = time.time()
    model = LinearRegression()
    model.fit(X_train_normalized, y_train_normalized)

    # Predict on the test set
    y_pred_test = model.predict(X_test_normalized)
    # Denormalize predictions
    y_pred_test_denorm = y_pred_test * close_std + close_mean

    # # Apply the condition to y_test and y_pred_test_denorm
    y_test = np.where(y_test >= 0.02, 1, 0)
    y_pred_test_classified = np.where(y_pred_test_denorm >= 0.02, 1, 0)

    # Track the end time of the process
    end_time = time.time()


# Convert results to a DataFrame for further analysis
results_df = pd.DataFrame({
    'y_test': y_test,
    'y_pred_test_classified': y_pred_test_classified
})

# Print the results DataFrame
results_df

Processing train-test split ratio: 0.93


,y_test,y_pred_test_classified
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
2532,0,1
2533,0,1
2534,0,1
2535,0,1


In [66]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

# Calculate Accuracy
accuracy = accuracy_score(results_df['y_test'], results_df['y_pred_test_classified'])

# Calculate F1 Score
f1 = f1_score(results_df['y_test'], results_df['y_pred_test_classified'])

# Print results
print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.85
F1 Score: 0.73


In [67]:
from sklearn.metrics import accuracy_score, f1_score

# Take the last 100 rows
results_df_last_100 = results_df.tail(100)

# Calculate Accuracy
accuracy = accuracy_score(results_df_last_100['y_test'], results_df_last_100['y_pred_test_classified'])

# Calculate F1 Score
f1 = f1_score(results_df_last_100['y_test'], results_df_last_100['y_pred_test_classified'])

# Print results
print(f"Accuracy (Last 100): {accuracy:.2f}")
print(f"F1 Score (Last 100): {f1:.2f}")

Accuracy (Last 100): 0.84
F1 Score (Last 100): 0.84


In [9]:
# # Example: Take the last 100 rows
# y_pred_test_denorm_last_100 = y_pred_test_denorm[-100:]

# # Convert to a DataFrame
# df = pd.DataFrame(y_pred_test_denorm, columns=['prediction_value'])

# # Save to CSV
# output_file = 'pred_value_20241712.csv'
# df.to_csv(output_file, index=False)

# print(f"Saved the last 100 rows to '{output_file}'")

Saved the last 100 rows to 'pred_value_20241712.csv'
